# Module 6 & 7 - Associations

## Notes
- Install these packages:

    + `pip3 install mlxtend`
    + `pip3 install openpyxl`

- Training data set is from here: 
    
    + `https://archive.ics.uci.edu/ml/datasets/online+retail`  
    + `http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx`

- Code example is from here: 
    + `https://pbpython.com/market-basket-analysis.html`
    
- Support & Confidence:

    Support, $s(X \to Y) = \frac{\sigma(X \cup Y)}{N}$
    
    Confidence, $c(X \to Y) = \frac{\sigma(X \cup Y)}{\sigma (X)}$
    
    Where, $X \cap Y = \emptyset $

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt

from mlxtend.preprocessing import TransactionEncoder

from mlxtend.frequent_patterns import apriori

from itertools import permutations, combinations

In [ ]:
df = pd.read_excel('../data/Online_Retail.xlsx')
# df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
init_series = df.copy()
df.head()

## Notes
Remove items that lack invoice number (Null/NaN) or have letter `C` as `credit transactions`.


In [ ]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
print('initial length:', len(init_series))
print('    new length:', len(df))
print(' items removed:', len(init_series)-len(df))

In [ ]:
basket = (df[df['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

In [ ]:
len(rules)

In [ ]:
new_rules = rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]
new_rules

In [ ]:
len(new_rules)

In [ ]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

In [ ]:
basket['ALARM CLOCK BAKELIKE RED'].sum()


In [ ]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]


# Example #2

`https://medium.com/analytics-vidhya/association-analysis-in-python-2b955d0180c`



In [ ]:

df = pd.read_csv('../data/retail_dataset.csv')
dataset = df.to_numpy()
df = df.replace(np.nan, '', regex=True)
df

In [ ]:

dataset = df.to_numpy()


new_dataset = []
for row in dataset:
    new_row = []
    for i in row:
        if i != '':
            new_row.append(i)
    new_dataset.append(new_row)

te = TransactionEncoder()
te_ary = te.fit(new_dataset).transform(new_dataset)
df1 = pd.DataFrame(te_ary, columns=te.columns_)    
df1    

In [ ]:
freq_items = apriori(df1, min_support=0.25, use_colnames=True, verbose=1)
freq_items

In [ ]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
rules.head()
print(len(rules))

In [ ]:
plt.scatter(rules['support'], rules['confidence'], alpha=0.5)
plt.xlabel('support')
plt.ylabel('confidence')
plt.title('Support vs Confidence')
plt.show()

In [ ]:
plt.scatter(rules['support'], rules['lift'], alpha=0.5)
plt.xlabel('support')
plt.ylabel('lift')
plt.title('Support vs Lift')
plt.show()

In [ ]:
fit = np.polyfit(rules['lift'], rules['confidence'], 1)
plt.xlabel('lift')
plt.ylabel('confidence')
fit_fn = np.poly1d(fit)
plt.plot(rules['lift'], rules['confidence'], 'yo', rules['lift'], fit_fn(rules['lift']))

# Homework Assignment # 3

1. Run the entire code below with these two datasets:

```python
T = [['a','b'], ['c','d'], ['a','c'], ['a','b','d'], ['a','e'], ['d'], ['a'], ['b']]
T = [['Bread', 'Milk'], 
     ['Bread', 'Diapers', 'Beer', 'Eggs'], 
     ['Milk', 'Diapers', 'Beer', 'Cola'],
     ['Bread', 'Milk', 'Diapers', 'Beer'],
     ['Bread', 'Milk', 'Diapers', 'Cola']]

```
2. Don't run the congressional vote dataset. It is too large (36 variables)

3. Verify the results of the program manually using the algorithms provided. 

4. Write your report and submit. Use either Jupyter notebook or Word or both.


In [ ]:
issues = ['','handicapped-infants','water-project-cost-sharing',
          'adoption-of-the-budget-resolution',
          'physician-fee-freeze',
          'el-salvador-aid',
          'religious-groups-in-schools',
          'anti-satellite-test-ban',
          'aid-to-nicaraguan-contras',
          'mx-missile',
          'immigration',
          'synfuels-corporation-cutback',
          'education-spending',
          'superfund-right-to-sue',
          'crime',
          'duty-free-exports',
          'export-administration-act-south-africa']
len(issues)

## Loading congressional votes 1984.

In [ ]:
df = pd.read_csv('../data/house-votes-84.data')

## Making Transaction set from a dataframe

In [ ]:
def make_T_from_dataframe(df):
    new_T = []
    for index, row in df.iterrows():
        t = []
        for item in row:
            if len(item) > 0:
                t.append(item)
        new_T.append(t)
    return new_T

In [ ]:
def get_84_congressional_votes(df):
    new_T = []
    for index, row in df.iterrows():
        t = []
        j = 0
        for item in row:
            if item == 'y' or item == 'n':
                # t.append(issues[j]+'--'+item)
                t.append(f'{j}_{item}')
            elif item != '?':
                t.append(item)
            j += 1
        new_T.append(t)
    return new_T


## Loading a Transaction sets

In [ ]:
T = [['a','b'], ['c','d'], ['a','c'], ['a','b','d'], ['a','e'], ['d'], ['a'], ['b']]
T = [['Bread', 'Milk'], 
     ['Bread', 'Diapers', 'Beer', 'Eggs'], 
     ['Milk', 'Diapers', 'Beer', 'Cola'],
     ['Bread', 'Milk', 'Diapers', 'Beer'],
     ['Bread', 'Milk', 'Diapers', 'Cola']]

# df is loading elsewhere above
# T = make_T_from_dataframe(df) 

# T = get_84_congressional_votes(df)

sorted_T = []
for t in T:
    t.sort()
    sorted_T.append(t)
T = sorted_T    
list(T)
print(f'There are {len(T)} transactions:')
display(T)

In [ ]:
# Making Universe set U from transaction set T
def make_U(T):    
    u = {}
    for t in T:
        for i in t:
            u[i] = i
    
    U = list(u)
    U.sort()
    return U

U = make_U(T)

print(f'There are {len(U)} items:')
display(U)

# Brute-force
Printing out a lattice of n items of universe U

In [ ]:
"""
Generate all possible k-itemsets for a universe U
Organize the list by row, with row 0 contains null,
row 1 contains all 1-itemsets, row 2 contains all 2-itemsets, etc.
"""
def bf_all_itemsets(U):
    bf_candidates = [[]]
    print(U)
    for c_len in range(1,len(U)+1):
        cs = combinations(U,c_len)
        row = []
        for c in cs:
            c = list(c)
            c.sort()
            row.append(c)

        bf_candidates.append(row)
    return bf_candidates


## Check to see if an item is in an itemset

In [ ]:
"""
return whether or not a k_itemset is in a transaction t.
"""
def is_k_itemset_in_transaction(k_itemset, t):
    return all(item in t for item in k_itemset)

## Get all the FI candidates that meet or exceed the support count based on a set of transaction

In [ ]:
"""
For a given set of k-itemsets, with at support count against a transaction T,
return a set of frequent k-itemsets.
"""

def get_candidates(sup_count, k_itemsets, T):
    k_itemset_coll = {}
    
    """
    First, we count the occurence of a k-itemset in the set of 
    transaction T.
    """
    for k_itemset in k_itemsets:
        k_itemset_key = '_'.join(k_itemset)
        for t in T:
            if is_k_itemset_in_transaction(k_itemset, t):
                if k_itemset_key in k_itemset_coll:
                    item = k_itemset_coll[k_itemset_key]
                    item[1] += 1
                    k_itemset_coll[k_itemset_key] = item
                else:
                    item = [k_itemset, 1]
                    k_itemset_coll[k_itemset_key] = item           

    """
    Then, we are looking for k-itemsets that is infrequent.
    We keep the keys of the removal k-itemsets in a list
    We then go through the removal list and remove the 
    k-itemsets by key
    """                
    remove_keys = []
    for i in k_itemset_coll:
        item = k_itemset_coll[i]
        if item[1] < sup_count:
            remove_keys.append(i)
    for key in remove_keys:
        del k_itemset_coll[key]
        
    return k_itemset_coll



## Displaying all itemsets

In [ ]:
all_itemsets = bf_all_itemsets(U)
# display(all_itemsets)

### Displaying the candidates and support count

In [ ]:
for sup_count in range(1, len(T)+1):
    
    minsup = sup_count/len(T)
    print(f'\n*** supp_count:{sup_count}, minsup:{minsup}\n')

    candidates = [[]] # Null first entry as in lattice      

    for i in range(1,len(all_itemsets)):
        candidates.append(get_candidates(sup_count, all_itemsets[i], T))

    for i in range(1,len(candidates)):
        print(f'{i}({len(candidates[i])}): {candidates[i]}\n')

# Apriori algorithm
Implement Apriori algorithm ...

$F_{k} = {i\ |\ i \in I \land \sigma({i}) \ge N \ x \ minsup}$

## Step 1: Generating `k` itemsets from Universal U

In [ ]:
"""
Generating a set of all possible k-itemsets transactions T 
from a set of universal items U
"""
def make_T(a_U, k):
    cs = combinations(a_U, k)
    k_itemsets = []
    for c in cs:
        c = list(c)
        c.sort()
        k_itemsets.append(c)
    return k_itemsets

### Extract items out of a collection of k-itemsets

In [ ]:
def get_k_itemsets(ck):
    k_itemsets = []
    for item in ck:
        k_itemsets.append(ck[item][0])
    return k_itemsets

### Prune

In [ ]:
"""
Pruning. starting with 2. If k-1 subset of k_candidate 
is not a part of k_minus_1_candidate, drop the k-itemset.
"""
def candidate_prune(kminus1_candidates, k_candidates):
    print('(k-1)-itemsets:')
    display(kminus1_candidates)
    print('k-itemsets:')
    display(k_candidates)
    
    return_k_candidates = []
    
    for i in range(len(k_candidates)):        
        k = len(k_candidates[i])        
        if k == 1:
            print('*** Nothing to prune ***')
            return k_candidates
            
        k_candidate = [k_candidates[i]]
        a_U = make_U(k_candidate)
        kminus1_subsets = make_T(a_U, k-1)
        if all(item in kminus1_candidates for item in kminus1_subsets):
            return_k_candidates.append(k_candidates[i])
            
    return return_k_candidates

### Generating Candidates

In [ ]:
"""
Generating all possible k-itemset transactions T
from a previous set of (k-1)-itemsets 
"""
def candidate_gen(kminus1_itemsets, k):  
    return make_T(make_U(kminus1_itemsets), k)

### The apriori algorithm.

In [ ]:
"""
"""    
def apriori_gen(U, T, sup_count):    
    UFk = [[]]
    a_U = U
    # display(a_U)
    a_T = make_T(a_U, 1)
    # display(a_T)

    prev_ck = a_T
    for k in range (1, 6):
        print(f'*** generating {k}-itemsets ***')
        #k_itemsets = make_T(a_U, k)
        k_itemsets = candidate_gen(prev_ck, k)
        
        print(f'*** pruning {k}-itemsets ***')
        # print('** before pruning **')
        # display(k_itemsets)
        k_itemsets = candidate_prune(prev_ck, k_itemsets)        
        # print('** after pruning **')
        # display(k_itemsets)

        print('*** verifying frequent itemsets ***')
        ck = get_candidates(sup_count, k_itemsets, T)
        display(ck)
        # k_itemsets = get_k_itemsets(ck)
        prev_ck = get_k_itemsets(ck)

        if ck == None or len(ck) == 0:
            print ('** DONE **')
            break

        UFk.append(ck)
        k_itemsets = get_k_itemsets(ck)
        # a_U = make_U(k_itemsets)     
    return UFk


In [ ]:
sup_count = 2
UFk = apriori_gen(U, T, sup_count)

print('*** UFk ***')
display(UFk)

In [ ]:
# Computing Confidence
# for each frequent k-itemset fk, k >= 2 do
#   H1 = 1
# end for

for k in range(1, len(UFk)):
    for key in UFk[k]:
        k_itemset = UFk[k][key][0]
        k_count = UFk[k][key][1]
        print(key, k_itemset, k_count)
        h1 = []
        for item in k_itemset:
            h1.append([item])
        print(h1)


        

In [ ]:
# itemset = ['a','b','c']
all_rules = {}
for k in range(2, len(UFk)):
    for key in UFk[k]:
        k_itemset = UFk[k][key][0]
        k_count = UFk[k][key][1]
        print('\n\n***', key, k_itemset, k_count)
        h1 = []

        super_set = []
        for count in range(1,len(k_itemset)):
            for c in combinations(k_itemset, count):
                value_key = '_'.join(c)
                print(key, value_key, c, UFk[len(c)][value_key][1])
                super_set.append(c)
        
        sig_a_b = k_count
        rules 
        for i in range(len(super_set)-1):
            a = super_set[i]
            for j in range(i+1, len(super_set)):
                b = super_set[j]
                if set(a).isdisjoint(b) and len(a) + len(b) == len(k_itemset):
                    sig_a = UFk[len(a)]['_'.join(a)][1]
                    sig_b = UFk[len(b)]['_'.join(b)][1]                                        
                    all_rules['_'.join(a) + '->' + '_'.join(b)] = sig_a_b/sig_a
                    all_rules['_'.join(b) + '->' + '_'.join(a)] = sig_a_b/sig_b
                    

In [ ]:
print(f'there are {len(all_rules)} rules')
rules

In [ ]:
print(f'number of transaction in the dataset is {len(T)}')

In [ ]:
confidence = 0.97
adopted_rules = {}
for key in all_rules:
    if all_rules[key] >= confidence:
        adopted_rules[key] = all_rules[key]

In [ ]:
print(f'there are {len(adopted_rules)} adopted rules of a {confidence} of confidence or greater')

for key in adopted_rules:
    if key.startswith('adoption-of-the-budget-resolution_mx-missile') and key.endswith('->democrat'):
        print(key, adopted_rules[key])

## Congressional Voting Record examples
